## Homework 2: Glow

Напомним, что нормализующие потоки — это класс генеративных моделей, которые строят сложную, но при этом обратимую функцию $\boldsymbol{f}_{\boldsymbol{\theta}}$, отображающую некоторое простое распределение в сложное распределения наших данных. Обучая эту функцию, мы сможем генерировать новые данные и оценивать их плотность $p_{\boldsymbol{\theta}}(\mathbf{x})$.

Это становится возможным благодаря теореме о замене переменных, которая лежит в основе нормализующих потоков. Она позволяет связать плотность $p_{\boldsymbol{\theta}}(\mathbf{x})$ с плотностью базового распределения $p(\mathbf{z}$) через обратимую функцию $\boldsymbol{f}_{\boldsymbol{\theta}}$
и её якобиан:

$$p_{\boldsymbol{\theta}}(\mathbf{x}) = p(\boldsymbol{f}_{\boldsymbol{\theta}}(\mathbf{x})) \bigg| \det\left(\frac{\partial\boldsymbol{f_{\boldsymbol{\theta}}}​}{\partial \mathbf{x}} \right) \bigg|$$

Обычно функция $\boldsymbol{f}_{\boldsymbol{\theta}}$ — это не простое преобразование, а сложная композиция нескольких обратимых функций:

$$\boldsymbol{f}_{\boldsymbol{\theta}} = \boldsymbol{f}_{K, \boldsymbol{\theta}}\circ \dots\circ \boldsymbol{f}_{1,\boldsymbol{\theta}}$$
​
Поскольку детерминант якобиана всей композиции равен произведению детерминантов якобианов каждого преобразования, мы можем переписать формулу в следующем виде:

$$p_{\boldsymbol{\theta}}(\mathbf{x}) = p(\mathbf{z}_K) \left| \prod_{k=1}^K \det\left(\frac{\partial \boldsymbol{f}_{k,\boldsymbol{\theta}}}{\partial \mathbf{z}_{k-1}} \right) \right|$$

<center><img src="images/flow.png" width=700></center>

На семинаре мы познакомились с архитектурой **RealNVP** и увидели две главные особенности модели — **Affine Coupling Layers** и **Multi-Scale архитектуру**.

Однако, как и в любой работе, в **RealNVP** оставались моменты, которые можно было улучшить. Именно этим и занялись исследователи из `OpenAI` и предложили модель [Glow](https://arxiv.org/pdf/1807.03039). Она берет лучшее от RealNVP и улучшает слабые места:

- Вместо фиксированной перестановки использует `обратимую 1x1 свертку`, которая позволяет модели самой находить оптимальный способ перемешивания каналов.

- Вместо стандартного `BatchNorm`, который плохо работает с маленькими батчами, использует `ActNorm`.

### Задание

Вам предстоит реализовать модель `Glow` и обучить модель на датасете `CelebA` для решения задачи генерации лиц.

За выполнение домашнего задания можно получить до **10 баллов**. Для части заданий мы написали для вас скелет. Заполните в них пропуски, выделенные с помощью `...`.

In [ ]:
import math
import os
from typing import List, Optional, Tuple
import zipfile

from IPython.display import clear_output
import matplotlib.pyplot as plt
from PIL import Image
import requests
import torch
import torch.distributions as D
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
from torchvision.utils import make_grid
from tqdm.auto import tqdm

### Задание 1: Dataset (0.5 балла)

Для обучения нашей модели мы будем использовать датасет `CelebA`, который содержит более $200$ тыс. изображений лиц знаменитостей. Поскольку стандартные средства torchvision для его загрузки часто нестабильны, мы скачаем архив с изображениями напрямую и напишем датасет самостоятельно.

**Ваша задача**:

- Скачать и распаковать архив с датасетом

- Создать преобразования для всех изображений: 
    - обрезать до квадрата по центру размером $148\times148$ (`CenterCrop`), 
    - измененить размера до $64\times64$ (`Resize`) 
    - преобразовать в тензор (`ToTensor`).

- Реализовать и создать класс CelebADataset.

- Разделить созданный датасет на обучающую и валидационную выборки в соотношении $90\% / 10\%$

- Создать DataLoader'ы для каждой выборки.

In [ ]:
def download_and_unzip(url, save_path, extract_path, chunk_size=128):
    if os.path.exists(extract_path):
        print(f"Directory '{extract_path}' already exists. Skipping download.")
        return
    
    print(f"Downloading archive from {url}...")
    r = requests.get(url, stream=True)
    with open(save_path, 'wb') as fd:
        for chunk in tqdm(r.iter_content(chunk_size=chunk_size)):
            fd.write(chunk)
    print("Download complete.")
    
    print(f"Unzipping archive '{save_path}'...")
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(os.path.dirname(extract_path))
    print("Unzipping complete.")
    os.remove(save_path)
    print(f"Archive '{save_path}' deleted.")
    
DATASET_URL = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip" 
ZIP_PATH = "celeba.zip"
DATA_ROOT = "data/celeba"
IMAGE_DIR = os.path.join(DATA_ROOT, "images")

download_and_unzip(DATASET_URL, ZIP_PATH, IMAGE_DIR)

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_files = sorted(os.listdir(root_dir))
        self.transform = transform

    def __len__(self):
        ...
    def __getitem__(self, idx):
        ...


In [ ]:
generator = torch.Generator().manual_seed(42)   # for train_test split

#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

Давайте взглянем на наши данные.

In [ ]:
indices = torch.randperm(len(train_dataset))[:40]

fig, axes = plt.subplots(5, 8, figsize=(16, 10))

for i, ax in enumerate(axes.flat):
    image, label = train_dataset[indices[i]]  

    image = image.permute(1, 2, 0).cpu().numpy()

    ax.imshow(image)
    ax.axis('off')

plt.tight_layout()
plt.show()

### Задание 2: ActNorm (1 балл)

Одним из важных нововведений модели Glow является слой `ActNorm`. Как и `BatchNorm`, он выполняет аффинное преобразование $y = s\cdot x + b$ для каждого канала, чтобы стабилизировать и ускорить обучение. Однако параметры масштаба (`scale`) и сдвига (`bias`) в `ActNorm` не зависят от текущего батча, а являются обучаемыми параметрами модели.

При этом есть небольшая особенность в их инициализации, которая **зависит от данных**:

- При первом проходе данных через модель (на самом первом батче), параметры `scale` и `bias` вычисляются таким образом, чтобы выходные активации для каждого канала имели нулевое среднее и единичную дисперсию.

- После такой инициализации, они становятся обычными параметрами и обновляются с помощью градиентного спуска, как и все остальные веса сети.

Такой подход делает `ActNorm` независимым от размера батча, что важно для обучения на больших изображениях, где размер батча часто приходится делать очень маленьким.

Как и любой слой в нормализующем потоке, `ActNorm` должен быть обратимым и иметь легко вычисляемый детерминант якобиана. Для преобразования $\mathbf{z}=\mathbf{s}\odot\mathbf{x}+\mathbf{b}$, логарифм детерминанта имеет вид:

$$\log|\det(\mathbf{J}^{Actnorm})| = h \cdot w \cdot \sum_{c=1}^C(\log|s_c|)$$

​
Здесь $s_c$ — это параметр масшта для $c$-го канала, $h\cdot w$ — количество пикселей в изображении.

Вам необходимо реализовать класс `ActNorm`. В конструкторе уже определены параметры `scale`, `bias` и флаг `initialized`.

Ваша задача — заполнить пропуски в методах `forward` и `inverse`.
​

In [ ]:
class ActNorm(nn.Module):
    def __init__(self, num_channels: int):
        super().__init__()
        self.scale = nn.Parameter(torch.ones(1, num_channels, 1, 1))
        self.bias = nn.Parameter(torch.zeros(1, num_channels, 1, 1))
        
        # Register a buffer to track whether the layer has been initialized.
        self.register_buffer("initialized", torch.tensor(False, dtype=torch.bool))

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Data-dependent initialization: this block runs only once, on the first forward pass.
        if not self.initialized:
                
            # Calculate mean and std for each channel
            mean = ...
            std = ...
                
            # Initialize scale and bias using the calculated statistics.
            # To make the output have mean=0, std=1, we need b = -mean and s = 1/std.
            ...
                
            # Mark the layer as initialized
            ...

        # Apply the forward affine transformation
        z = ...
        
        # Calculate the log-determinant of the Jacobian for the forward pass.
        logdet = ...
        
        return z, logdet

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        # Apply the inverse transformation.
        x = ...
        
        return x

### Задание 3: Invertible 1x1 Convolution (1.5 балла)

В **слоях связи** (**Affine Coupling Layers**) за один шаг преобразование затрагивает только половину каналов. Чтобы модель была мощной, информация между этими шагами должна "перемешиваться", позволяя всем каналам влиять друг на друга. В RealNVP для этого использовалась фиксированная перестановка каналов. Это работало, но было неоптимально, так как модель не могла "научиться" лучшему способу перемешивания.

В **Glow** для этого предложили использовать **обратимую свертку $1\times 1$** (`Invertible 1x1 Convolution`). С точки зрения математики, свертка $1\times1$ с матрицей весов $\mathbf{W}$ размера $C\times C$ эквивалентна умножению вектора каналов C на эту матрицу для каждого пикселя в изображении. Таким образом, эта свертка и есть наше обучаемое перемешивание.

Основная сложность здесь — вычислить детерминант якобиана. Для свертки $1\times1$ якобиан — это и есть матрица весов $\mathbf{W}$. Прямое вычисление $\det(\mathbf{W})$ имеет сложность $\mathcal{O}(C^3)$. Чтобы избежать этого, авторы **Glow** параметризуют матрицу $\mathbf{W}$ через ее `LU-разложение`:

$$\mathbf{W}=\mathbf{P}\mathbf{L}(\mathbf{U}+diag(\mathbf{s}))$$

где $\mathbf{P}$ — матрица перестановки, $\mathbf{L}$ — нижнетреугольная матрица, $\mathbf{U}$ — верхнетреугольная матрица с нулями на диагонали, $diag(\mathbf{s})$ - диагональная матрица, у которой на диагонали стоят элементы вектора $\mathbf{s}$, а в остальных местах — нули.

В результате мы получим, что 

$$\log|\det(\mathbf{J}^{Conv1x1 ​ })| = h \cdot w \cdot \log |\det (\mathbf{W})|$$

$$\log|\det(\mathbf{W})| = \sum_{c=1}^C \log|s_c|,$$

где $s_c$ — диагональные элементы матрицы $\mathbf{U}$.

В этом задании вам предстоит реализовать слой `Invertible1x1Conv`.

Для выполнения задания рекомендуем ознакомиться с [`LU-разложением`](https://docs.pytorch.org/docs/stable/generated/torch.linalg.lu.html) в `Pytorch`.

In [ ]:
class Invertible1x1Conv(nn.Module):
    def __init__(self, num_channels: int):
        super().__init__()
        
        # Initialize with a random orthogonal matrix
        w_init = torch.linalg.qr(torch.randn(num_channels, num_channels))[0]
        
        # Perform LU-decomposition
        ...
        
        # Extract diagonal elements 's' from U
        ...
        
        # Register P (permutation), sign_s, L and U masks as non-trainable buffers
        ...

        # Define L, U (without diagonal) and log_s as learnable parameters
        ...
        
    def _calculate_weight(self, inverse: bool) -> torch.Tensor:
        
        # Reconstruct L from the learnable part self.l and an identity matrix
        l = ...
        
        # Reconstruct U
        u = ...
        
        if inverse:
            w_inv = ...
            return w_inv
        else:
            w = ...
            return w

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        
        # Get the weight matrix
        weight = ...
        
        # Apply the 1x1 convolution
        z = ...
        
        # Calculate the log-determinant
        logdet = ...
        
        return z, logdet

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        # Get the inverse weight matrix
        weight_inv = ...
        
        # Apply the inverse 1x1 convolution
        x = ...
        
        return x

### Задание 4: Affine Coupling Layers (1 балл)

Основой как RealNVP, так и Glow, является **афинный слой связи** (**Affine Coupling Layer**). Его дизайн был настолько удачен, что в Glow он остался без изменений. 

Напомним, что в таких слоях входной вектор делился на две части $\mathbf{x}_{1:d}$ и $\mathbf{x}_{d+1:D}$, а латентный вектор $\mathbf{z}$ получался следующим образом:

$$\begin{cases} \mathbf{z}_{1:d} &= \mathbf{x}_{1:d} \\ \mathbf{z}_{d+1:D} &= \mathbf{x}_{d+1:D}\odot e^{s(\mathbf{x}_{1:d})} + t(\mathbf{x}_{1:d}) \end{cases}$$

Благодаря такому преобразованию, искомый определитель якобиана вычислялся очень просто:

$$\det(\mathbf{J}) = \prod_{j=1}^{D-d}e^{s(\mathbf{x}_{1:d})_j} = e^{\sum_{j=1}^{D-d} s(\mathbf{x}_{1:d})_j}$$

$$\log|\det(\mathbf{J}^{Coupling})| = \sum_{j=1}^{D-d} s(\mathbf{x}_{1:d})_j$$

В этом задании вам предстоит реализовать слой `AffineCouplingLayer`. В конструкторе уже определена нейросеть `self.net`, которая будет вычислять параметры `s` и `t`.

In [ ]:
class AffineCouplingLayer(nn.Module):
    def __init__(self, num_channels: int, hidden_channels: int = 512):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(num_channels // 2, hidden_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_channels, hidden_channels, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(hidden_channels, num_channels, kernel_size=3, padding=1)
        )
        # We initialize the last layer with zeros, so the whole coupling layer starts
        # as a near-identity function (z ≈ x), which ensures a stable start to training.
        self.net[-1].weight.data.zero_()
        self.net[-1].bias.data.zero_()

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Split the input tensor into two halves along the channel dimension
        x_a, x_b = ... 
        
        # Pass one half through the network to get the parameters for the other half and get log_s and t
        log_s, t = ...
        
        # Get s
        s = ...
    
        #  Apply the affine transformation to the other half
        z_b = ...
        
        # Concatenate the transformed half and the unchanged half
        z = ... 
        
        #  Calculate the log-determinant
        logdet = ...
        
        return z, logdet

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        # Split the input tensor z into two halves
        z_a, z_b = ...
        
        # Pass the unchanged half to get the parameters
        log_s, t = ...
        s = ...
        
        # Apply the inverse affine transformation
        x_b = ...
        
        # Concatenate the result with the unchanged half
        x = ...
        
        return x

### Задание 5: Squeeze (0.5 балла)

Операция **сжатия** (**Squeeze**) является важной особенностью **Multi-Scale архитектуры**, унаследованной от `RealNVP`. Её задача — изменить форму тензора, преобразуя пространственные размерности в канальные.

В этом задании вам нужно реализовать прямое и обратное преобразование для слоя `Squeeze`.

In [ ]:
class Squeeze(nn.Module):
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        
        # Reshape the tensor to factor out 2x2 blocks from spatial dimensions
        x = ...
        
        # Permute the dimensions to bring the 2x2 factors next to the channel dimension
        x = ...
        
        # Reshape again to merge the new dimensions with the channel dimension.
        x = ...
        
        return x

    def inverse(self, z: torch.Tensor) -> torch.Tensor:

        # Reshape to factor out the channel dimension back into 2x2 blocks.
        z = ...
        
        # Permute the dimensions to move the 2x2 blocks back to the spatial dimensions.
        z = ...
        
        # Reshape again to get the final output.
        z = ...
        return z

### Задание 6: Split and GaussianPrior (1 балл)

**Multi-Scale архитектура** `Glow/RealNVP` работает по принципу **factoring out**. На каждом уровне (кроме последнего) мы **отщепляем** половину каналов, превращая их в латентные переменные $\mathbf{z}$, а оставшуюся часть отправляем снова в модель. За это отвечают слои `Split` и `GaussianPrior`.

- Задача `Split`разделить входной тензор $\mathbf{x}$ на две половины, $\mathbf{x}_1$ и $\mathbf{x}_2$, затем передать их в `GaussianPrior` для обработки и собирать результаты.

- `GaussianPrior` принимает $\mathbf{x}_1$ как условие и $\mathbf{x}_2$ как таргет. С помощью нейросети он предсказывает параметры `mean` и `log_std` нормального распределения для $\mathbf{x}_2$, основываясь на $\mathbf{x}_1$. Затем он использует эти параметры, чтобы выполнить преобразование из $\mathbf{x}_2$ в $\mathbf{z}_2$ и вычислить `logdet`.

<center><img src="images/factor_out.png" width=250></center>

Важная операция внутри `GaussianPrior` — это стандартизация. Её цель — выполнить обратимое преобразование из $\mathbf{x}_2$ в $\mathbf{z}_2$ так, чтобы $\mathbf{z}_2$ подчинялся простому стандартному нормальному распределению $\mathcal{N}(\mathbf{0},\mathbf{I})$:

$$\mathbf{z}= \frac{\mathbf{x} - \boldsymbol{\mu}}{\boldsymbol{\sigma}}$$

**Заметка об обозначениях:** В описании выше мы использовали букву $\mathbf{x}$ для простоты и общности. Однако важно понимать, что операция `Split` применяется **на каждом уровне Multi-Scale архитектуры**, а не только к исходному изображению. Поэтому в коде мы будем использовать букву $\mathbf{h}$ (**hidden**), которая обозначает скрытое состояние на промежуточных слоях.

**Hint**:
Деление на `std` эквивалентно умножению на $e^{-log\_std}$.

Вам необходимо реализовать классы `GaussianPrior` и `Split`, заполнив пропуски в коде.

In [ ]:
class GaussianPrior(nn.Module):
    def __init__(self, num_channels: int):
        super().__init__()
        self.net = nn.Conv2d(num_channels, 2 * num_channels, kernel_size=3, padding=1)
        # Initialize with zeros for a stable start
        self.net.weight.data.zero_()
        self.net.bias.data.zero_()

    def forward(self, h1: torch.Tensor, h2: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Predict parameters for the distribution of h2, conditioned on h1.
        mean, log_std = ...
        
        # Transform h2 into z2 using the standardization formula.
        z2 = ...
        
        # Calculate the log-determinant
        logdet = ...
        
        return z2, logdet
    
    def inverse(self, h1: torch.Tensor, z2: torch.Tensor) -> torch.Tensor:
        # Predict parameters from h1, just like in the forward pass.
        mean, log_std = ...
        
        # Perform the inverse transformation to generate h2 from the noise z2.
        h2 = ...
        
        return h2

In [ ]:
class Split(nn.Module):
    def __init__(self, num_channels: int):
        super().__init__()
        
        self.prior = GaussianPrior(num_channels // 2)

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        # Split the input tensor into two halves
        x1, x2 = ...
        
        # Delegate the transformation of x2 to the prior.
        z2, logdet = ... 
        
        return x1, z2, logdet

    def inverse(self, x1: torch.Tensor, z2: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Delegate the inverse transformation to the prior.
        x2 = ... 
        
        # Concatenate x1 and the reconstructed x2.
        x = ... 
        
        return x

### Задание 7: Preprocess (0.5 балла)

Как вы знаете, нормализующие потоки по своей природе работают с непрерывными данными, в то время как изображения состоят из дискретных пикселей от $0$ до $255$. Чтобы подружить модель с данными, мы будем применять специальный слой предобработки, который выполняет две функции:

1. **Деквантизация** (**Dequantization**): Здесь мы превращаем дискретные значения в непрерывные. Стандартный способ — добавить к каждому значению пикселя равномерный шум из распределения $\mathcal{U}(0, \frac{1}{256})$. Это "размывает" каждое дискретное значение, что делает данные совместимыми с моделью.

2. **Центрирование** (**Centering**): Большинство нейронных сетей лучше обучаются, когда их входные данные центрированы вокруг нуля, поэтому мы сдвигаем диапазон данных из $[0, 1]$ в $[-0.5, 0.5]$.

Весь этот процесс является обратимым аффинным преобразованием. А значит, мы обязаны вычислить логарифм детерминанта его якобиана, чтобы корректно посчитать итоговое правдоподобие. Для масштабирования на $\frac{1}{256}$ `logdet` на один пиксель равен $\log(\frac{1}{256})=−\log(256)$.

In [ ]:
class Preprocess(nn.Module):
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:        
        # Dequantize the data by adding uniform noise.
        x = ...
        
        # Calculate the log-determinant for this transformation.
        logdet = ... 
        
        # Center the data from [0, 1] to [-0.5, 0.5]
        z = x - 0.5
        
        return z, logdet

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        
        # Un-center the data from [-0.5, 0.5] back to [0, 1]
        x = ...
        
        return x

### Задание 8: Flow Step (0.5 балла)

До этого момента мы реализовывали отдельные обратимые преобразования. Теперь наша задача — объединить их в единый блок `Flow Step`, который и будет составлять основу нашей нейросети. В модели `Glow` каждый `FlowStep` представляет собой композицию из трех последовательных преобразований, которые мы уже реализовали:

1. `ActNorm`

2. `Invertible1x1Conv`

3. `AffineCouplingLayer`

Вам необходимо реализовать класс `FlowStep`, который последовательно применяет слои `ActNorm`, `Invertible1x1Conv` и `AffineCoupling`.

In [ ]:
class FlowStep(nn.Module):
    def __init__(self, num_channels: int, hidden_channels: int):
        super().__init__()
        self.actnorm = ...
        self.conv = ...
        self.coupling = ...

    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        
        # Apply the transformations in the correct forward order and accumulate the log-determinants
        x, logdet_act = ...
        x, logdet_conv = ...
        x, logdet_coupling = ...
        
        # Sum the log-determinants from each step
        total_logdet = ...
        return x, total_logdet

    def inverse(self, z: torch.Tensor) -> torch.Tensor:
        
        # Apply the inverse transformations in the reverse order.
        z = ...
        return z

### Задание 9: MultiScaleBlock (1 балл)

Основная идея **MultiScale архитектуры** — обработка данных на разных масштабах. На каждом уровне модель анализирует текущее представление данных, **отщепляет** (**factor out**) часть информации, которую она уже смогла смоделировать, а оставшуюся часть передает дальше.

`MultiScaleBlock` состоит из трех последовательных этапов, которые мы уже реализовали:

1. `Squeeze`: Пространственное разрешение уменьшается, а глубина каналов увеличивается.

2. `K шагов FlowStep`: К новому представлению применяется композиция из $K$ обратимых преобразований.

3. `Split`: Половина обработанных каналов **отщепляется** и превращается в латентные переменные $\mathbf{z}$, другая половина передается на вход следующему `MultiScaleBlock`.

Этот процесс повторяется несколько раз. Самый последний блок в модели не выполняет `Split`, так как передавать оставшиеся данные уже некуда.

Вам необходимо реализовать класс `MultiScaleBlock`, который объединяет ранее созданные вами слои.

In [ ]:
class MultiScaleBlock(nn.Module):
    def __init__(self, num_channels: int, num_flows: int, hidden_channels: int, split: bool = True):
        super().__init__()
        
        self.squeeze = Squeeze()
        
        # Calculate the number of channels after the Squeeze operation
        squeezed_channels = ... 
        
        # Initialize a list of FlowStep layers
        self.flow_steps = ...
        
        # Initialize the Split layer if required
        self.split = split
        if self.split:
            self.split_layer = ...
        
    def forward(self, x: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor, Optional[torch.Tensor]]:
        """
        Returns:
        - x_out: The tensor to be passed to the next level.
        - logdet_sum: The total log-determinant from all operations in this block.
        - z_split: The factored-out latent z, or None if split=False.
        """
        x = self.squeeze(x)
        
        # Initialize the log-determinant accumulator for this block
        logdet_sum = ...
        
        # Apply the sequence of FlowSteps
        for flow in self.flow_steps:
            # Get the output tensor and the logdet from the flow step
            x, logdet = ...
            logdet_sum = ...
            
        z_split = None
        # Apply the Split layer if this is not the last block
        if self.split:
            # The Split layer returns the ongoing tensor x, the factored-out z, and its logdet
            x, z, logdet = ...
            logdet_sum = ...
            z_split = z
            
        return x, z_split, logdet_sum

    def inverse(self, x_next: torch.Tensor, z_split: Optional[torch.Tensor]) -> torch.Tensor:
        """
        Accepts:
        - x_next: The output from the next (deeper) block.
        - z_split: The factored-out z that was created by this block during the forward pass.
        """
        
        # Apply the inverse Split operation first (if it exists) to reconstruct the tensor
        if self.split:
            # Reconstruct the full tensor before the Squeeze operation
            x = ...
        else:
            x = ...

        # Apply the inverse of FlowSteps in reverse order
        for flow in reversed(self.flow_steps):
            x = ...
            
        # Apply the inverse Squeeze operation
        x = ...
        
        return x

### Задание 10: Glow (1 балл)

Нам остался последний шаг — собрать все компоненты вместе.

Вам предстоит собрать финальный класс `Glow`. Все необходимые блоки у вас уже есть.

#### Bits Per Dimension

Когда мы обучаем генеративные модели, мы максимизируем логарифм правдоподобия $\log p_{\boldsymbol{\theta}}(\mathbf{x})$. Однако сами по себе эти значения не очень интуитивны. Что значит $\log p_{\boldsymbol{\theta}}(\mathbf{x}) = 1500$? Это хороший или плохой результат? Более того, это значение сильно зависит от размера изображения — для картинки $64\times 64$ оно будет гораздо ниже, чем для $32\times 32$, что мешает сравнивать модели.

Чтобы решить эти проблемы, используется более интерпретируемая метрика — **биты на размерность** (**Bits Per Dimension**, **BPD**).

**BPD** — это среднее количество бит, которое требуется нашей модели для кодирования одной размерности данных (одного цветового канала одного пикселя).

Чем ниже BPD, тем лучше. Низкий BPD означает, что модель хорошо выучила распределение данных, считает их очень вероятными и может "сжать" их с минимальными затратами.

Оптимальное количество бит, необходимое для кодирования события с вероятностью $p(\mathbf{x})$, равно $-\log p(\mathbf{x})$. Наша модель вычисляет $\ln p(\mathbf{x})$. Чтобы перейти к логарифму по основанию $2$, используется простая формула $\log_2 p(\mathbf{x})= \frac{\ln p(\mathbf{x})}{ln(2)}$. Таким образом, общее число бит для кодирования всего изображения равно $-\frac{\ln p(\mathbf{x})}{ln(2)}$.

Чтобы получить среднее значение на размерность, мы делим общее число бит на количество всех размерностей и получаем финальную формулу для **BPD**:

$$BPD = -\frac{\ln p(\mathbf{x})}{ln(2)\cdot h\cdot w\cdot c}$$

**Hint**:

Исходные 8-битные цветные изображения требуют ровно $8$ бит на каждую размерность. Модель, которая не выучила ничего, будет иметь **BPD** около $8.0$. 

In [ ]:
class Glow(nn.Module):
    """
    The final, complete Glow model, built using the MultiScaleBlock abstraction.
    """
    def __init__(self, input_shape: Tuple[int, int, int], num_levels: int, num_flows_per_level: int, hidden_channels: int):
        super().__init__()
        
        self.preprocess = Preprocess()
        self.blocks = nn.ModuleList()
        
        C, H, W = input_shape
        current_channels = C
        for i in range(num_levels):
            is_last_block = (i == num_levels - 1)
            # Append a MultiScaleBlock to the self.blocks list, use the `split` argument to control whether it's the last block.
            self.blocks.append(
                ...
            )
            # Update the number of channels for the next block.
            if not is_last_block:
                current_channels = ...

        # Calculate and store the final shape of z
        self.final_z_shape = self._calculate_final_z_shape(input_shape, num_levels)
        
        self.register_buffer('base_dist_mean', torch.zeros(1))
        self.register_buffer('base_dist_var', torch.ones(1))

    def _calculate_final_z_shape(self, input_shape: Tuple[int, int, int], num_levels: int) -> Tuple[int, int, int]:
        C, H, W = input_shape
        for i in range(num_levels):
            H //= 2
            W //= 2
            C *= 4
            
            if i < num_levels - 1:
                C //= 2
        return C, H, W

    @property
    def base_dist(self):
        return D.Normal(self.base_dist_mean, self.base_dist_var)

    def forward(self, x: torch.Tensor) -> Tuple[List[torch.Tensor], torch.Tensor]:
        zs = []
        # Apply the preprocessing layer and get the initial logdet
        x, logdet_sum = ...

        for block in self.blocks:
            # Pass x through the block and get the outputs
            x, z_split, logdet = ...
            
            # Accumulate the log-determinant
            logdet_sum = ...
            
            # If z_split is not None, add it to the list of zs
            if z_split is not None:
                ...
        
        # Append the final x (which is the last z) to the list
        ...
        
        return zs, logdet_sum

    def inverse(self, batch_size: int, z_std: float = 1.0) -> torch.Tensor:
        device = self.base_dist_mean.device
        
        # Start by sampling z from the base distribution
        C, H, W = self.final_z_shape
        x = self.base_dist.sample((batch_size, C, H, W)).squeeze(-1).to(device) * z_std
        
        # Loop through the blocks in reverse order
        for block in reversed(self.blocks):
            # If the block had a split, we need to generate a z_split to merge with x
            if block.split:
                # Generate z_split on the fly, with the same shape as x
                z_split = ...
                x = ...
            else:
                x = ...
        
        # Apply the inverse of the preprocessing layer
        x = ...
        
        return x

    def log_prob(self, x: torch.Tensor, bits_per_pixel: bool = True) -> torch.Tensor:
       
        # Get the latent variables and the total log-determinant from the forward pass
        zs, logdet = ...
        
        # Calculate the log-probability of each z under the base distribution.
        base_log_prob = ...
        
        # Combine everything using the Change of Variables formula.
        log_prob = ...

        if bits_per_pixel:
            # Convert log-likelihood to bits per dimension
            bpd = ...
            return bpd
            
        return log_prob


### Задание 11: Sampling, Training and Validation Loop (0.5 балла)

Теперь, когда у нас есть полностью собранная модель Glow, нам нужно ее обучить. Процесс обучения, валидации и генерации реализован в нескольких вспомогательных функциях. Ваша задача - заполнить недостающие части кода.

In [ ]:
@torch.no_grad()
def sample(model, n_samples=5, z_std=0.7):
    model.eval()
    samples = ...
    return samples

In [ ]:
def train(model, optimizer, train_loader, device, grad_clip=None):
    model.train()
    total_loss = 0.0

    for batch_x, _ in tqdm(train_loader, desc="Train", leave=False):
        x = ...

        loss = ...

        if grad_clip is not None:
            torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)

        total_loss += ...

    return ...


@torch.no_grad()
def validate(model, val_loader, device):
    model.eval()
    total_loss = 0.0

    for batch_x, _ in tqdm(val_loader, desc="Val", leave=False):
        x = ...
        loss = ...
        total_loss += ...

    return ...

In [ ]:
def plot_losses(train_losses, val_losses):
    clear_output(wait=True)
    plt.figure(figsize=(6, 4))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Bits per dim')
    plt.title('Loss Curve')
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_images(images, nrow=5, title=None):
    if images.min() < 0:
        images = images - images.min()
        images = images / images.max()

    grid = make_grid(images, nrow=nrow, padding=2)
    np_img = grid.permute(1, 2, 0).cpu().numpy()

    plt.figure(figsize=(nrow * 2, (len(images) // nrow + 1) * 2))
    if title:
        plt.title(title)
    plt.imshow(np_img)
    plt.axis('off')
    plt.show()

def save_checkpoint(model, optimizer, epoch, path_template):
    save_path = path_template.format(epoch=epoch)
    save_dir = os.path.dirname(save_path)
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)

    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, save_path)

    print(f"Checkpoint saved: {save_path}")

In [ ]:
def train_and_validate(model, optimizer, train_loader, val_loader, num_epochs, device,
                       grad_clip=None, checkpoint_path=None):
    model = model.to(device)

    train_losses, val_losses = [], []

    for epoch in range(1, num_epochs + 1):
        train_loss = ...
        val_loss = ...

        print(f"[Epoch {epoch}/{num_epochs}] Train: {train_loss :.4f} | Val: {val_loss:.4f}")
        
        plot_losses(train_losses, val_losses)

        samples = sample(model, n_samples=10, z_std=0.7)
        plot_images(samples, nrow=5, title=f"Epoch {epoch} samples")

        if checkpoint_path:
            save_checkpoint(model, optimizer, epoch, checkpoint_path)

Итак, мы реализовали все основные блоки, необходимые для обучения модели `Glow`. Теперь осталось собрать все воедино, задать гиперпараметры и запустить процесс обучения. Вы можете экспериментировать с различными гиперпараметрами, чтобы добиться хороших результатов.

In [ ]:
L = 3
K = 32
INPUT_SHAPE = (3, 64, 64)
HIDDEN_CHANNELS = 512

model = Glow(input_shape=(3, 64, 64), num_levels=L, num_flows_per_level=K, hidden_channels=HIDDEN_CHANNELS)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
train_and_validate(
    model,
    optimizer,
    train_loader,
    val_loader,
    num_epochs=50,
    device=device,
    sample_every=1,
    grad_clip=3,
    checkpoint_path="checkpoints_glow/epoch_{epoch}.pth"
)

### Задание 12: Sampling Temperature (0.5 балла)

После того как мы обучили модель, мы можем загрузить её, чтобы использовать для генерации. Код ниже загружает веса модели, которые были сохранены во время обучения.

In [ ]:
checkpoint_path = "checkpoints_glow/epoch_30.pth"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Model loaded from: {checkpoint_path}")

model.to(device)

Теперь начинается самое интересное — исселедовать процесс генерации. Какой параметр, по вашему мнению, отвечает за температуру генерации? Каким образом это происходит?

**Ваш ответ:**

Проверьте вашу гипотезу на практике. Сгенерируйте по $20$ изображений c различными значениями этого параметра и сделайте выводы.

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

### Задание 13: Latent Space (0.5 балла)

До сих пор мы рассматривали нашу модель как "черный ящик", который при обратном проходе возвращает изображение. Но самое интересное происходит в его латентном пространстве.

**Латентное пространство** — это многомерное векторное пространство, в котором модель хранит свое понимание данных в некотором структурированном виде. 

В плохой, необученной модели это пространство хаотично. Но хорошо обученная модель организует его осмысленно.

**Свойства "хорошего" латентного пространства**:

- **Гладкость**: Точки, которые находятся близко друг к другу в латентном пространстве, должны соответствовать визуально похожим изображениям. Если мы возьмем вектор $\mathbf{z}$ и немного его сдвинем, получив $\mathbf{z}'$, то сгенерированное изображение $\mathbf{x}'$ должно лишь слегка отличаться от исходного $\mathbf{x}$.

- **Семантическая структура**: Модель часто учится выстраивать в пространстве целые направления, соответствующие осмысленным атрибутам. Например, движение вдоль одной оси может делать лицо на картинке более улыбчивым, а движение вдоль другой — изменять цвет волос.

Мы можем проверить, что наша модель действительно выучила такое гладкое и осмысленное пространство с помощью линейной интерполяции:

1. Мы берем два разных изображения, например, мужское лицо ($\mathbf{x}_{man}$) и женское ($\mathbf{x}_{woman}$).

2. Находим их вектора в латентном пространстве: $\mathbf{z}_{man}= f(\mathbf{x}_{man})$ и $\mathbf{z}_{woman}= f(\mathbf{x}_{woman})$

3. Поскольку $\mathbf{z}$ — это просто векторы, мы можем найти любую точку на прямой линии между ними по формуле:

$$\mathbf{z}_{interp} =  (1-\alpha)\cdot \mathbf{x}_{man} + \alpha \cdot \mathbf{x}_{woman}$$

Если пространство действительно гладкое и осмысленное, то, декодируя эти промежуточные точки $\mathbf{z}_{interp}$, мы должны увидеть плавный правдоподобный переход одного изображения в другое.

В этом задании вам предстоит реализовать две новые функции:

- `inverse_reconstruct`: декодер, который принимает на вход конкретный список латентных векторов `zs` и восстанавливает из них изображение.

- `interpolate_glow`: функция для интерполяции, которая будет использовать `inverse_reconstruct` для визуализации плавного перехода между двумя изображениями.

In [ ]:
def inverse_reconstruct(model, zs: List[torch.Tensor]) -> torch.Tensor:
    """
    Performs the inverse pass from a given list of latent tensors `zs`.
    """
    
    #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
    # Your code here
    
    return ...

def interpolate_glow(model, image1: torch.Tensor, image2: torch.Tensor, num_steps: int, device: torch.device) -> torch.Tensor:
    """
    Performs latent space interpolation between two images using the Glow model.
    """
    model.eval()
    x1 = image1.unsqueeze(0).to(device)
    x2 = image2.unsqueeze(0).to(device)

    # Encode the two images to get their latent representations using the model's forward pass.
    zs1, _ = ...
    zs2, _ = ...

    # For simplicity, we will only interpolate the last (deepest) z-vector in the list.
    z1_deepest = zs1[-1]
    z2_deepest = zs2[-1]
    
    # Create a tensor of alpha values for interpolation (from 0.0 to 1.0).
    alphas = ...
    
    interpolated_images = []
    for alpha in tqdm(alphas, desc="Generating interpolation"):
        # Calculate the interpolated z-vectors 
        ...
        
    return ...

Найдите в валидационном датасете пары изображений, соответствующие описаниям ниже, и, используя реализованные вами функции для каждой пары выполните интерполяцию с `num_steps=10`:

- **Мужчина** $\rightarrow$ **Женщина**
- **Улыбающееся лицо** $\rightarrow$ **Нейтральное лицо**
- **Светлые волосы** $\rightarrow$ **Темные волосы**


In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ 
# Your code here

In [ ]:
# Здесь можно оставить отзывы, пожелания и впечатления о ДЗ :)